In [1]:
import pandas as pd
import numpy as np
import glob
import os


In [2]:
path = '../data/raw' 

# Use glob to get a list of all csv files in the folder
all_files = glob.glob(os.path.join(path, "*.csv")) # <-- Optional but good practice

# Create a list to hold the individual dataframes
li = []

for filename in all_files:
    # Read each csv file into a dataframe
    df_temp = pd.read_csv(filename, index_col=None, header=0)
    
    # --- THIS IS THE CORRECTED LINE ---
    # It correctly handles file paths on any operating system (Windows, Mac, Linux)
    brand = os.path.basename(filename).split('.')[0]
    
    # Create a new 'brand' column
    df_temp['brand'] = brand
    
    # Append the dataframe to the list
    li.append(df_temp)

# Concatenate all dataframes in the list into a single dataframe
df = pd.concat(li, axis=0, ignore_index=True)

# Display the first 5 rows to see the combined data
print("First 5 rows of the combined dataset:")
display(df.head())

# Display a summary of the combined dataframe
print("\nCombined Dataset Information:")
df.info()

First 5 rows of the combined dataset:


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,brand,tax(£),fuel type,engine size,mileage2,fuel type2,engine size2,reference
0,A1,2017.0,12500,Manual,15735,Petrol,150.0,55.4,1.4,audi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A6,2016.0,16500,Automatic,36203,Diesel,20.0,64.2,2.0,audi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A1,2016.0,11000,Manual,29946,Petrol,30.0,55.4,1.4,audi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A4,2017.0,16800,Automatic,25952,Diesel,145.0,67.3,2.0,audi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A3,2019.0,17300,Manual,1998,Petrol,145.0,49.6,1.0,audi,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Combined Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118150 entries, 0 to 118149
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   model         117995 non-null  object 
 1   year          117903 non-null  float64
 2   price         117995 non-null  object 
 3   transmission  117995 non-null  object 
 4   mileage       117077 non-null  object 
 5   fuelType      108540 non-null  object 
 6   tax           94327 non-null   float64
 7   mpg           99187 non-null   float64
 8   engineSize    108540 non-null  float64
 9   brand         118150 non-null  object 
 10  tax(£)        4860 non-null    float64
 11  fuel type     3517 non-null    object 
 12  engine size   9345 non-null    object 
 13  mileage2      9399 non-null    object 
 14  fuel type2    8537 non-null    object 
 15  engine size2  8537 non-null    object 
 16  reference     9455 non-null    object 
dtypes: float64(5), ob

In [3]:
# Check for missing values in each column
print(df.isnull().sum())

model              155
year               247
price              155
transmission       155
mileage           1073
fuelType          9610
tax              23823
mpg              18963
engineSize        9610
brand                0
tax(£)          113290
fuel type       114633
engine size     108805
mileage2        108751
fuel type2      109613
engine size2    109613
reference       108695
dtype: int64


In [4]:
# Clean up column names
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('£', '')

# Display the cleaned column names
print("Cleaned column names:")
print(df.columns)

Cleaned column names:
Index(['model', 'year', 'price', 'transmission', 'mileage', 'fueltype', 'tax',
       'mpg', 'enginesize', 'brand', 'tax', 'fuel_type', 'engine_size',
       'mileage2', 'fuel_type2', 'engine_size2', 'reference'],
      dtype='object')


In [5]:
# 1. Merge columns with similar meanings but slightly different names
# It takes the value from 'enginesize' if 'engine_size' is null, and so on.
df['engine_size'] = df['engine_size'].fillna(df['enginesize'])
df['fuel_type'] = df['fuel_type'].fillna(df['fueltype'])

# 2. Handle the duplicated 'tax' column
# This finds all columns named 'tax', takes the first available value from them for each row,
# and stores it in a new column called 'tax_final'.
tax_columns = df.columns[df.columns == 'tax']
df['tax_final'] = df[tax_columns].apply(
    lambda x: x.bfill().iloc[0] if x.notna().any() else None,
    axis=1
)

# 3. Define all original, duplicate, or unnecessary columns to be dropped
columns_to_drop = [
    'enginesize',          # Now merged into 'engine_size'
    'fueltype',            # Now merged into 'fuel_type'
    'tax',                 # The original duplicate columns
    'mileage2', 'fuel_type2', 'engine_size2', 'reference' # Unnecessary columns
]

df = df.drop(columns=columns_to_drop)

# 4. Rename 'tax_final' to the clean, final 'tax' column name
df.rename(columns={'tax_final': 'tax'}, inplace=True)

# 5. Check the result
print("Missing values after consolidating columns:")
print(df.isnull().sum())
print("\nFinal columns:")
print(df.columns)

Missing values after consolidating columns:
model             155
year              247
price             155
transmission      155
mileage          1073
mpg             18963
brand               0
fuel_type        6093
engine_size       265
tax             18963
dtype: int64

Final columns:
Index(['model', 'year', 'price', 'transmission', 'mileage', 'mpg', 'brand',
       'fuel_type', 'engine_size', 'tax'],
      dtype='object')


In [6]:

# Define the columns that should be numeric but might contain non-numeric characters
cols_to_clean = ['price', 'tax', 'mileage']

for col in cols_to_clean:
    # Use .str.replace() to remove the unwanted characters
    # We chain two .replace() calls: one for '£' and one for ','
    df[col] = df[col].astype(str).str.replace('£', '').str.replace(',', '')
    
    # Convert the cleaned column to a numeric type (float)
    # errors='coerce' will turn any values that still can't be converted into 'NaN' (missing values)
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("Successfully cleaned and converted columns to numeric type.")

# Check the data types to confirm the change
df[['price', 'tax', 'mileage']].info()

Successfully cleaned and converted columns to numeric type.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118150 entries, 0 to 118149
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   price    117995 non-null  float64
 1   tax      99187 non-null   float64
 2   mileage  117073 non-null  float64
dtypes: float64(3)
memory usage: 2.7 MB


In [7]:
cols_to_clean = ['price', 'tax', 'mileage']

for col in cols_to_clean:
    # First, check if the column's data type is 'object' (which is how pandas stores strings)
    if df[col].dtype == 'object':
        print(f"Found string column '{col}'. Cleaning now...")
        # If it is a string, remove unwanted characters
        df[col] = df[col].str.replace('£', '').str.replace(',', '')
        
        # Then, convert to a numeric type
        df[col] = pd.to_numeric(df[col], errors='coerce')

print("\nFinished checking and cleaning columns.")

# Verify the final data types
df[['price', 'tax', 'mileage']].info()


Finished checking and cleaning columns.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118150 entries, 0 to 118149
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   price    117995 non-null  float64
 1   tax      99187 non-null   float64
 2   mileage  117073 non-null  float64
dtypes: float64(3)
memory usage: 2.7 MB


In [8]:
# --- Final, Direct Approach to Data Type Correction ---

print("Attempting a direct conversion...")
cols_to_clean = ['price', 'tax', 'mileage']

for col in cols_to_clean:
    # Step 1: Force the entire column to be a string type. This is the key change.
    df[col] = df[col].astype(str)
    
    # Step 2: Now that it's guaranteed to be a string, perform replacements.
    df[col] = df[col].str.replace('£', '').str.replace(',', '')
    
    # Step 3: Convert the clean strings back to numbers.
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("\nConversion complete. Final data types:")
df[['price', 'tax', 'mileage']].info()

Attempting a direct conversion...

Conversion complete. Final data types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118150 entries, 0 to 118149
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   price    117995 non-null  float64
 1   tax      99187 non-null   float64
 2   mileage  117073 non-null  float64
dtypes: float64(3)
memory usage: 2.7 MB


In [10]:
# --- FINAL & DEFINITIVE DATA TYPE CORRECTION ---

# Define the columns that must be numeric
cols_to_convert = ['price', 'tax', 'mileage']

for col in cols_to_convert:
    # Force the column to string, remove all non-numeric characters (except the decimal point),
    # and convert to a numeric type.
    df[col] = df[col].astype(str).str.replace(r'[£,]', '', regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("Definitive cleaning and conversion complete.")
# Verify the final data types
df[['price', 'tax', 'mileage']].info()

Definitive cleaning and conversion complete.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118150 entries, 0 to 118149
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   price    117995 non-null  float64
 1   tax      99187 non-null   float64
 2   mileage  117073 non-null  float64
dtypes: float64(3)
memory usage: 2.7 MB


In [15]:
# --- FINAL DIAGNOSIS: CHECK ALL MODEL FEATURES ---

# Define all columns used in the machine learning model
features_to_check = ['year', 'price', 'mileage', 'engine_size', 'tax']

print("Checking all columns used in the model for conversion errors...")

error_found = False
for col in features_to_check:
    problem_values = []
    # Create a temporary series after cleaning
    cleaned_series = df[col].astype(str).str.replace('£', '').str.replace(',', '')
    
    # Loop through each unique value in the CLEANED series
    for value in cleaned_series.unique():
        try:
            float(value)
        except (ValueError, TypeError):
            problem_values.append(value)
            
    if problem_values:
        print(f"\n!!!!!! Found problematic values in column: '{col}' !!!!!!")
        print(problem_values)
        error_found = True

if not error_found:
    print("\nNo conversion errors found in any of the feature columns. The issue may be elsewhere.")

Checking all columns used in the model for conversion errors...

!!!!!! Found problematic values in column: 'engine_size' !!!!!!
['2148CC', ' 200 cc']


In [16]:
# --- NEW CELL: Clean 'engine_size' Column ---

# Convert to string and use a regular expression to extract only the first sequence of numbers
# For example, '2148CC' becomes '2148'
df['engine_size'] = df['engine_size'].astype(str).str.extract(r'(\d+)').astype(float)

# Impute any values that might have become null during this process (e.g., if a value had no numbers)
df['engine_size'].fillna(df['engine_size'].median(), inplace=True)

print("Successfully cleaned the 'engine_size' column.")

Successfully cleaned the 'engine_size' column.


C:\Users\azhar\AppData\Local\Temp\ipykernel_9556\1640773865.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['engine_size'].fillna(df['engine_size'].median(), inplace=True)


In [19]:
# --- FINAL & DEFINITIVE IMPUTATION ---

# Define all columns used in the machine learning model
features = ['year', 'price', 'mileage', 'engine_size', 'tax', 'mpg']

# Loop through each feature and fill any missing values with the median
for col in features:
    if df[col].isnull().any():
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)
        print(f"Filled missing values in '{col}' with median value: {median_val:.2f}")

# Impute remaining categorical columns just in case
categorical_cols = ['model', 'transmission', 'fuel_type']
for col in categorical_cols:
    if df[col].isnull().any():
        mode_val = df[col].mode()[0]
        df[col].fillna(mode_val, inplace=True)

# --- FINAL CHECK ---
# Verify that there are no more missing values in the entire dataframe
print("\nFinal check for missing values:")
print(df.isnull().sum())

C:\Users\azhar\AppData\Local\Temp\ipykernel_9556\630176583.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_val, inplace=True)
C:\Users\azhar\AppData\Local\Temp\ipykernel_9556\630176583.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

Filled missing values in 'year' with median value: 2017.00
Filled missing values in 'price' with median value: 14960.00
Filled missing values in 'mileage' with median value: 17154.00
Filled missing values in 'tax' with median value: 145.00
Filled missing values in 'mpg' with median value: 54.30

Final check for missing values:
model           0
year            0
price           0
transmission    0
mileage         0
mpg             0
brand           0
fuel_type       0
engine_size     0
tax             0
dtype: int64


In [21]:
# --- CORRECTED MODEL TRAINING WORKFLOW ---
from sklearn.linear_model import LinearRegression

# Step 1: Do a final drop of any rows where critical info might still be missing after cleaning
df.dropna(subset=['year', 'price', 'mileage', 'engine_size', 'tax', 'mpg'], how='any', inplace=True)

# Step 2: Separate the data based on 'mpg' BEFORE final imputation
train_df = df[df['mpg'].notna()].copy()
predict_df = df[df['mpg'].isna()].copy()

# Step 3: Define features and target
features = ['year', 'price', 'mileage', 'engine_size', 'tax']
target = 'mpg'

# Step 4: Impute missing values in the feature columns for both dataframes
for col in features:
    median_val = train_df[col].median() # Calculate median only from training data
    train_df[col].fillna(median_val, inplace=True)
    predict_df[col].fillna(median_val, inplace=True) # Use same median for prediction set

# Step 5: Train the model
X_train = train_df[features]
y_train = train_df[target]

model = LinearRegression()
model.fit(X_train, y_train)

# Step 6: Predict and fill (if there's anything to predict)
if not predict_df.empty:
    X_predict = predict_df[features]
    predicted_mpg = model.predict(X_predict)
    df.loc[df['mpg'].isna(), 'mpg'] = predicted_mpg
    print("Successfully imputed 'mpg' using a predictive model!")
else:
    print("No 'mpg' values needed predictive imputation.")

# --- FINAL OVERALL IMPUTATION FOR ANY OTHER MISSING VALUES ---
for col in df.columns:
    if df[col].dtype == 'object':
        df[col].fillna(df[col].mode()[0], inplace=True)
    else:
        df[col].fillna(df[col].median(), inplace=True)

print("\nFinal check for missing values:")
print(df.isnull().sum().sum()) # Should be 0

C:\Users\azhar\AppData\Local\Temp\ipykernel_9556\3341310866.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df[col].fillna(median_val, inplace=True)
C:\Users\azhar\AppData\Local\Temp\ipykernel_9556\3341310866.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

No 'mpg' values needed predictive imputation.

Final check for missing values:
0


C:\Users\azhar\AppData\Local\Temp\ipykernel_9556\3341310866.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
C:\Users\azhar\AppData\Local\Temp\ipykernel_9556\3341310866.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

In [22]:
# Now, let's clean up the rest using the simple median/mode method
# Impute remaining numerical values with their respective medians
numerical_cols = ['year', 'price', 'mileage', 'engine_size', 'tax']
for col in numerical_cols:
    if df[col].isnull().any():
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)

# Impute remaining categorical values with their respective modes
# Note: 'model' and 'transmission' might still have nulls from the original file issues
categorical_cols = ['model', 'transmission', 'fuel_type']
for col in categorical_cols:
    if df[col].isnull().any():
        mode_val = df[col].mode()[0]
        df[col].fillna(mode_val, inplace=True)

# --- FINAL CHECK ---
# Verify that there are no more missing values
print("\nFinal check for missing values:")
print(df.isnull().sum())


Final check for missing values:
model           0
year            0
price           0
transmission    0
mileage         0
mpg             0
brand           0
fuel_type       0
engine_size     0
tax             0
dtype: int64
